### 3. Explore and cluster the neighborhood in Toronto

#### # Use geopy library to get the latitude and longitude values of Toronto City.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 3-1. Load the saved Totondo neighborhood dataset

In [41]:
## Load the saved dataset
df_toronto_org = pd.read_csv('toronto.csv')
df_toronto=df_toronto_org[['Borough','Neighbourhood','Latitude','Longitude']]
df_toronto.columns=['Borough','Neighborhood','Latitude','Longitude']
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


#### # Get the latitude & longitude value of Toronto city 

In [42]:
address = 'Toronto city'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### # Create a map of Toronto with neighborhoods superimposed on top.

In [43]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(df_toronto, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### # Define [Foursquare] Credentials and Version

In [44]:
CLIENT_ID = 'K0LM422ISMHBBHHPHTWR0C13NJ51GDYHG2VQERH4WS335140' # your Foursquare ID
CLIENT_SECRET = 'XSVGW3FVARC3UZT5V30RPE4FQHWYWIYXHXCVDMCYQUG43W41' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K0LM422ISMHBBHHPHTWR0C13NJ51GDYHG2VQERH4WS335140
CLIENT_SECRET:XSVGW3FVARC3UZT5V30RPE4FQHWYWIYXHXCVDMCYQUG43W41


In [45]:
df_toronto.loc[0, 'Neighborhood']

'Malvern, Rouge'

 -- Get the neighborhood's latitude and longitude values.

In [46]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8066863, -79.19435340000003.


#### # Get the top 100 venues that are in Malvern, Rouge within a radius of 500 M

In [47]:
LIMIT=100
radius=500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
         CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, 
         VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=K0LM422ISMHBBHHPHTWR0C13NJ51GDYHG2VQERH4WS335140&client_secret=XSVGW3FVARC3UZT5V30RPE4FQHWYWIYXHXCVDMCYQUG43W41&ll=43.8066863,-79.19435340000003&v=20180605&radius=500&limit=100'

 -- Send the GET request and examine the resutls

In [48]:
results = requests.get(url).json()
#results

In [49]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

 -- Ready to clean the json and structure it into a *pandas* dataframe.

In [50]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [51]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### 3-2. Explore Neighborhoods in Toronto

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### # Write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [166]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                 latitudes=df_toronto['Latitude'],
                                 longitudes=df_toronto['Longitude'])
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [53]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, Richmond, King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, Steeles East, L'Amoreaux East, Milliken",3,3,3,3,3,3
"Alderwood, Long Branch",11,11,11,11,11,11
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,53,53,53,53,53,53
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CFB Toronto, Downsview East",3,3,3,3,3,3


In [54]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


### 3-3. Analyze Each Neighborhood

In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

In [56]:
toronto_onehot.shape

(2253, 275)

#### # Group rows by neighborhood and by taking the mean of the frequency of each category

In [167]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped.head()

In [58]:
toronto_grouped.shape

(101, 275)

#### # Print each neighborhood along with the top '3' most common venues

In [168]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

 -- Put the values into a *pandas* dataframe

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Gym,Restaurant,Cosmetics Shop,Hotel,Bar
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt North, Steeles East, L'Amoreaux East...",Coffee Shop,Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Alderwood, Long Branch",Pizza Place,Gym,Bank,Pharmacy,Pool,Pub,Dance Studio,Sandwich Place,Skating Rink,Coffee Shop
4,Bayview Village,Bank,Japanese Restaurant,Chinese Restaurant,Café,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [171]:
#toronto_grouped.head()

In [63]:
toronto_grouped.shape

(101, 275)

### 3-4. Cluster Neighborhoods

 -- Run *k*-means to cluster the neighborhood into 5 clusters.

In [126]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 3, 3, 0, 0, 3, 4, 4], dtype=int32)

In [145]:
torgr = toronto_grouped['Neighborhood'].reset_index(drop=True)
tordf = df_toronto['Neighborhood'].reset_index(drop=True)
diff= pd.concat([torgr,tordf]).drop_duplicates(keep=False)
diff

16         Upper Rouge
93    Islington Avenue
Name: Neighborhood, dtype: object

In [156]:
# df.loc[~df.b.isin(my_list)]
#(op) df_toronto2 = df_toronto[df_toronto.Neighborhood != ['Upper Rouge','Islington Avenue']]
df_toronto2 = df_toronto.loc[~df_toronto.Neighborhood.isin(diff)]
df_toronto2.reset_index(drop=True)
df_toronto2.shape

(101, 4)

 -- create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [157]:
#toronto_merged = df_toronto[['Borough','Neighbourhood','Latitude','Longitude']]
toronto_merged=df_toronto2

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Print Shop,Fast Food Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,History Museum,Moving Target,Bar,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
2,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711,4,Rental Car Location,Medical Center,Mexican Restaurant,Breakfast Spot,Electronics Store,Pizza Place,Donut Shop,Doner Restaurant,Dog Run,Dance Studio
3,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Indian Restaurant,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
4,Scarborough,Cedarbrae,43.773136,-79.239476,3,Fried Chicken Joint,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Thai Restaurant,Lounge,Hakka Restaurant,Concert Hall,Dessert Shop


In [158]:
toronto_merged.shape

(101, 15)

 -- visualize the resulting clusters

In [160]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3-5. Examine Clusters

 ==> Cluster 1

In [161]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Print Shop,Fast Food Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
5,Scarborough Village,Spa,Playground,Business Service,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
6,"Ionview, Kennedy Park, East Birchmount Park",Discount Store,Bus Station,Train Station,Department Store,Hobby Shop,Coffee Shop,Comfort Food Restaurant,Comic Shop,Empanada Restaurant,Electronics Store
10,"Scarborough Town Centre, Wexford Heights, Dors...",Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Diner,Dance Studio,Deli / Bodega,Department Store
12,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
14,"Agincourt North, Steeles East, L'Amoreaux East...",Coffee Shop,Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
15,"L'Amoreaux West, Steeles West",Chinese Restaurant,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Sandwich Place,Nail Salon,Breakfast Spot,Coffee Shop,Pharmacy,Grocery Store
17,Hillcrest Village,Pool,Mediterranean Restaurant,Dog Run,Athletics & Sports,Golf Course,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
18,"Fairview, Oriole, Henry Farm",Clothing Store,Fast Food Restaurant,Coffee Shop,Bus Station,Restaurant,Bakery,Cosmetics Shop,Food Court,Women's Store,Spa
19,Bayview Village,Bank,Japanese Restaurant,Chinese Restaurant,Café,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


 ==> Cluster 2

In [162]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,"Kingsway Park South West, The Queensway West, ...",Gym,Thrift / Vintage Store,Wings Joint,Fast Food Restaurant,Discount Store,Convenience Store,Burrito Place,Burger Joint,Sandwich Place,Bakery


 ==> Cluster 3

In [163]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,"Underground city, First Canadian Place",Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Gastropub,Deli / Bodega,Bar,Gym


 ==> Cluster 4

In [164]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",History Museum,Moving Target,Bar,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
3,Woburn,Coffee Shop,Korean Restaurant,Indian Restaurant,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
4,Cedarbrae,Fried Chicken Joint,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Thai Restaurant,Lounge,Hakka Restaurant,Concert Hall,Dessert Shop
7,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Soccer Field,Bus Station,Metro Station,Intersection,Ice Cream Shop,Construction & Landscaping,Convenience Store,Empanada Restaurant
11,"Wexford, Maryvale",Middle Eastern Restaurant,Shopping Mall,Auto Garage,Sandwich Place,Bakery,Breakfast Spot,Donut Shop,Doner Restaurant,Dog Run,Empanada Restaurant
13,"Sullivan, Clarks Corners, Tam O'Shanter",Pizza Place,Rental Car Location,Shopping Mall,Chinese Restaurant,Noodle House,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Women's Store
20,"Silver Hills, York Mills",Cafeteria,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Dance Studio
24,Willowdale West,Pharmacy,Butcher,Pizza Place,Coffee Shop,Gluten-free Restaurant,Cuban Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
26,Don Mills North,Pool,Caribbean Restaurant,Gym / Fitness Center,Japanese Restaurant,Basketball Court,Café,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
27,"Don Mills South, Flemingdon Park",Gym,Coffee Shop,Asian Restaurant,Beer Store,Chinese Restaurant,Dim Sum Restaurant,Italian Restaurant,Japanese Restaurant,Sporting Goods Shop,Fast Food Restaurant


 ==> Cluster 5

In [165]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Morningside, Guildwood, West Hill",Rental Car Location,Medical Center,Mexican Restaurant,Breakfast Spot,Electronics Store,Pizza Place,Donut Shop,Doner Restaurant,Dog Run,Dance Studio
8,"Cliffside, Scarborough Village West, Cliffcrest",Motel,Intersection,American Restaurant,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
9,"Cliffside West, Birch Cliff",Café,Skating Rink,College Stadium,General Entertainment,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
29,"Northwood Park, York University",Coffee Shop,Massage Studio,Furniture / Home Store,Metro Station,Bar,Caribbean Restaurant,Drugstore,Donut Shop,Dumpling Restaurant,Deli / Bodega
33,Downsview Northwest,Grocery Store,Discount Store,Athletics & Sports,Liquor Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
48,"Summerhill East, Moore Park",Restaurant,Park,Playground,Tennis Court,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
60,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Gym,Restaurant,Italian Restaurant,Gastropub,Sports Bar,Deli / Bodega
63,Roselawn,Ice Cream Shop,Garden,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
65,"Yorkville, The Annex, North Midtown",Coffee Shop,Sandwich Place,Café,Pizza Place,Cosmetics Shop,History Museum,Flower Shop,Indian Restaurant,BBQ Joint,Pub
78,"Exhibition Place, Parkdale Village, Brockton",Coffee Shop,Café,Breakfast Spot,Gym,Bakery,Furniture / Home Store,Italian Restaurant,Falafel Restaurant,Convenience Store,Performing Arts Venue


#### ** End of my practice session !! ** 